# Predicting using saved model

## Load Model

In [24]:
import tensorflow as tf
import numpy as np
from numpy import genfromtxt
import csv
import pandas as pd
import joblib
import tabulate
from sklearn.preprocessing import StandardScaler, MinMaxScaler

Load Model

In [14]:
saved_model_path = '/content/drive/MyDrive/jasakarya_saved_models/recsysnn_jasakarya_model_tf'

In [15]:
model = tf.keras.models.load_model(saved_model_path)

## Creating sample input for prediction

Load Target Jasa List (sampled from the training data)

In [16]:
file_path = '/content/drive/MyDrive/jasakarya_saved_models/synthetic_store_data_100k.csv'
jasa_data = np.genfromtxt(file_path, delimiter=',')

Shuffle the data

In [17]:
np.random.seed(42)
shuffled_data_index = np.random.permutation(jasa_data.shape[0])
subset_index = shuffled_data_index[:20]
sample_data_jasa = jasa_data[subset_index]
sample_jasa_vector = sample_data_jasa

Display the sample data

In [18]:
df = pd.DataFrame(sample_data_jasa)
print(df)

          0      1    2    3    4    5    6    7    8    9
0   75722.0  4.910  0.0  0.0  1.0  0.0  0.0  0.0  1.0  1.0
1   80185.0  4.052  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0
2   19865.0  2.086  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
3   76700.0  3.233  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0
4   92992.0  4.124  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0
5   76435.0  4.962  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0
6   84005.0  1.000  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7   80918.0  2.928  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
8   60768.0  2.242  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
9   50075.0  3.210  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
10  27702.0  4.876  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0
11  42142.0  3.015  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0
12  45081.0  5.000  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0
13  16639.0  4.072  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0
14  20425.0  2.867  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0
15  88703.0  3.066  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1

Create User Vector Input

In [19]:
sample_user_id    = 123456
sample_num_rating = 4
sample_avg_rating = 4.5
sample_cat_1      = 5.0
sample_cat_2      = 0.0
sample_cat_3      = 4.0
sample_cat_4      = 0.0
sample_cat_5      = 0.0
sample_cat_6      = 0.0
sample_cat_7      = 4.0
sample_cat_8      = 5.0

sample_user_vector = np.array([[sample_user_id, sample_num_rating, sample_avg_rating,
                                sample_cat_1,sample_cat_2,sample_cat_3,sample_cat_4,
                                sample_cat_5,sample_cat_6,sample_cat_7,sample_cat_8]])

Replicate user vector for every target item (jasa) given

In [20]:
sample_user_vectors = np.tile(sample_user_vector, (len(sample_jasa_vector), 1))

Load saved scalers

In [21]:
scalerUserLocation = "/content/drive/MyDrive/jasakarya_saved_models/scalerUser.joblib"
scalerJasaLocation = "/content/drive/MyDrive/jasakarya_saved_models/scalerJasa.joblib"
scalerTargetLocation = "/content/drive/MyDrive/jasakarya_saved_models/scalerTarget.joblib"

scalerUser = joblib.load(scalerUserLocation)
scalerJasa = joblib.load(scalerJasaLocation)
scalerTarget = joblib.load(scalerTargetLocation)

Scaling user and jasa vectors

In [22]:
sample_user_vectors = scalerUser.transform(sample_user_vectors)
sample_data_jasa = scalerJasa.transform(sample_jasa_vector)

Prediction

In [23]:
y_prediction = model.predict([sample_user_vectors[:,3:], sample_jasa_vector[:, 1:]])

1/1 [==============================] - 0s 142ms/step


Unscale prediction

In [25]:
y_prediction_unscaled = scalerTarget.inverse_transform(y_prediction)

Sorting and showing prediction

In [28]:
sorted_index = np.argsort(-y_prediction_unscaled, axis=0).reshape(-1).tolist() # Biggest score to lowest
sorted_prediction = y_prediction_unscaled[sorted_index]
sorted_jasa = sample_jasa_vector[sorted_index]

count = 0
display = [["y prediction", "jasa id", "jasa rating avg", "cat_1", "cat_2", "cat_3", "cat_4", "cat_5", "cat_6", "cat_7", "cat_8"]]

for i in range(0, sorted_prediction.shape[0]):
  if count == 15:
    break
  count += 1
  jasa_id = sorted_jasa[i].astype(int)
  display.append([np.around(sorted_prediction[i,0],1), sorted_jasa[i,0].astype(int), np.around(sorted_jasa[i,1].astype(float),1),
                  np.around(sorted_jasa[i,2].astype(float),1),np.around(sorted_jasa[i,3].astype(float),1),np.around(sorted_jasa[i,4].astype(float),1)
                  ,np.around(sorted_jasa[i,5].astype(float),1),np.around(sorted_jasa[i,6].astype(float),1),np.around(sorted_jasa[i,7].astype(float),1)
                  ,np.around(sorted_jasa[i,8].astype(float),1),np.around(sorted_jasa[i,9].astype(float),1)])

table = tabulate.tabulate(display, headers="firstrow")
print(table)

  y prediction    jasa id    jasa rating avg    cat_1    cat_2    cat_3    cat_4    cat_5    cat_6    cat_7    cat_8
--------------  ---------  -----------------  -------  -------  -------  -------  -------  -------  -------  -------
           4.7      84005                1          1        0        0        0        0        0        0        0
           4.3      19865                2.1        0        0        0        0        0        1        0        0
           4.3      60768                2.2        0        0        0        0        1        0        0        0
           4.2       1762                2.6        0        0        0        0        0        0        1        1
           4.2      20425                2.9        0        0        0        1        0        1        0        0
           4.2      42142                3          0        1        0        0        0        1        0        0
           4.2      80918                2.9        0        0  